<a href="https://colab.research.google.com/github/KathrynKashitsyn/LangChain-Agents-Social-Good/blob/main/Semantic_search_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Full tutorial https://python.langchain.com/docs/tutorials/retrievers/

# Abstractions

LangChain's *document loader*, *embedding*, and *vector store* abstractions are designed to support retrieval of data - from (vector) databases and other sources- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation, or RAG.

> Document loaders are designed to load document objects. LangChain has hundreds of integrations with various data sources to load data from: Slack, Notion, Google Drive, etc.

`from langchain_community.document_loaders.csv_loader import CSVLoader`


> Embedding models transform human language into a format that machines can understand and compare with speed and accuracy. These models take text as input and produce a fixed-length array of numbers, a numerical fingerprint of the text's semantic meaning.

`from langchain_openai import OpenAIEmbeddings`



> Vector stores are specialized data stores that enable indexing and retrieving information based on vector representations. They capture the semantic meaning of data that has been embedded.

`from langchain_core.vectorstores import InMemoryVectorStore`

# Setup

In [ ]:
pip install langchain-community pypdf

set environment variables to start logging traces

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


#Documents and Document Loaders

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has three attributes:

`page_content`: a string representing the content;

`metadata`: a dict containing arbitrary metadata;

`id`: (optional) a string identifier for the document.

The `metadata` attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual `Document` object often represents a chunk of a larger document.

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

LangChain ecosystem implements document loaders that integrate with hundreds of common sources. This makes it easy to incorporate data from these sources into the AI application.

#Loading documents

Let's load a PDF into a sequence of `Document` objects. There is a sample PDF in the LangChain repo here - a 10-k filing for Nike from 2025. We can consult the LangChain documentation for available PDF document loaders. Let's select PyPDFLoader, which is fairly lightweight.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "https://s1.q4cdn.com/806093406/files/doc_financials/2025/ar/Nike-Inc-2025_10K.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

102


`PyPDFLoader` loads one `Document` object per PDF page. For each, we can easily access:

- The string content of the page;
- Metadata containing the file name and page number.

In [ ]:
# [0]is a blank page, use [1]
print(f"{docs[1].page_content[:200]}\n")

SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K 
(Mark One)
☑ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
FOR THE FISCAL YEAR ENDED MAY 31,



In [ ]:
print(docs[0].metadata)

{'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.4 (Macintosh)', 'creationdate': '2025-07-23T08:45:44-04:00', 'moddate': '2025-07-23T23:08:01+08:00', 'source': 'https://s1.q4cdn.com/806093406/files/doc_financials/2025/ar/Nike-Inc-2025_10K.pdf', 'total_pages': 102, 'page': 0, 'page_label': '1'}


# Splitting

For both information retrieval and downstream question-answering purposes, a page may be too coarse a representation. Our goal in the end will be to retrieve `Document` objects that answer an input query, and further splitting our PDF will help ensure that the meanings of relevant portions of the document are not "washed out" by surrounding text.

We can use text splitters for this purpose. Here we will use a simple text splitter that partitions based on characters. We will split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the RecursiveCharacterTextSplitter, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index where each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

499

#Vector search

Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can embed it as a vector of the same dimension and use vector similarity metrics (such as cosine similarity) to identify related text.

LangChain supports embeddings from dozens of providers. These models specify how text should be converted into a numeric vector.

Let's select a model



In [ ]:
pip install -qU langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.01902569830417633, 0.03274717926979065, -0.01529537420719862, -0.024974869564175606, 0.013357077725231647, -0.008558119647204876, 0.035030487924814224, 0.054454315453767776, 0.02220456674695015, -0.011062843725085258]


#Vector stores

LangChain VectorStore objects contain methods for adding text and Document objects to the store, and querying them using various similarity metrics. They are often initialized with embedding models, which determine how text data is translated to numeric vectors.

LangChain includes a suite of integrations with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as Postgres) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads.

Let's select a vector store:

In [ ]:
pip install -qU langchain-core

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Having instantiated the vector store, we can now index the documents.

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

Once we've instantiated a `VectorStore` that contains documents, we can query it. VectorStore includes methods for querying:

Synchronously and asynchronously;
By string query and by vector;
With and without returning similarity scores;
By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).
The methods will generally include a list of Document objects in their outputs.

#Usage
Embeddings typically represent text as a "dense" vector such that texts with similar meanings are geometrically close. This lets us retrieve relevant information just by passing in a question, without knowledge of any specific key-terms used in the document.

Return documents based on similarity to a string query:

In [ ]:
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(results[0])

page_content='our Greater China geography, occupied by employees focused on implementing our wholesale, NIKE Direct and merchandising 
strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of 
which are owned and three of which are leased. Two other distribution centers, one located near Indianapolis, Indiana and one
located in Dayton, Tennessee, are leased and operated by third-party logistics providers. One distribution center for Converse is 
located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States, some of 
which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United 
States are located in Laakdal, Belgium; Taicang, China; Tomisato, Japan and Icheon, Korea, all of which we own.' metadata={'producer': 'Adobe PDF Library 17.0', 'creator':

Async query:

In [ ]:
results = await vector_store.asimilarity_search("When was Nike incorporated?")

print(results[0])

page_content='PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws of the State of Oregon. As used in this Annual Report on Form 10-K (this 
"Annual Report"), the terms "we," "us," "our," "NIKE" and the "Company" refer to NIKE, Inc. and its predecessors, subsidiaries 
and affiliates, collectively, unless the context indicates otherwise.
Our principal business activity is the design, development and worldwide marketing and selling of athletic footwear, apparel, 
equipment, accessories and services. NIKE is the largest seller of athletic footwear and apparel in the world. We sell our products 
through NIKE Direct operations, which are comprised of both NIKE-owned retail stores and sales through our digital platforms 
(also referred to as "NIKE Brand Digital") and to wholesale accounts, which include a mix of independent distributors, licensees 
and sales representatives in nearly all countries around the world. We also offer interactive consumer services and